# Seurat object  
## Load required packages


In [ ]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library(clustree)
library("future")



## Load seurat object (METABO)


In [ ]:
line <- params$line



In [ ]:
metabo_obj <- readRDS(file = paste0('./results/rds/samples/', line, '_metabo.rds'))

metabo_obj
table(metabo_obj$orig.ident)


In [ ]:
counts <- GetAssayData(metabo_obj, assay = "SCT")
print(length(rownames(counts)))

counts <- GetAssayData(metabo_obj, assay = "integrated")
print(length(rownames(counts)))

counts <- GetAssayData(metabo_obj, assay = "RNA")
print(length(rownames(counts)))


# Non linear reduction  
##  <span style="color:#296d98;"> Parameters </span>  


In [ ]:
pc = 45
8482 -> saved.seed
set.seed(saved.seed)
thread = 8



## UMAP


In [ ]:
metabo_obj <- RunUMAP(metabo_obj, reduction = "pca", dims = 1:pc, seed.use = saved.seed,verbose = FALSE )

DimPlot(metabo_obj, reduction = "umap", group.by = "orig.ident")
DimPlot(metabo_obj, reduction = "umap", group.by = "orig.ident", label.size = 4, label = TRUE)+ NoLegend()

DimPlot(metabo_obj, reduction = "umap", group.by = "Phase", cols = c(G1='azure4',G2M='dodgerblue', S='tomato3') )



### Split by condition


In [ ]:
DimPlot(metabo_obj, reduction = "umap", group.by="orig.ident", split.by = "condition")
DimPlot(metabo_obj, reduction = "umap", group.by = "orig.ident", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(metabo_obj, reduction = "umap", group.by="orig.ident", split.by = "condition")+ NoLegend()


## tSNE
tSNE with default Perplexity 30 :


In [ ]:
metabo_obj <- RunTSNE(metabo_obj, reduction = "pca", dims = 1:pc, seed.use = saved.seed,verbose = FALSE )

DimPlot(metabo_obj, reduction = "tsne", group.by = "orig.ident")
DimPlot(metabo_obj, reduction = "tsne", label.size = 4, label = TRUE, group.by = "orig.ident")

DimPlot(metabo_obj, reduction = "tsne", group.by = "Phase", cols = c(G1='azure4',G2M='dodgerblue', S='tomato3'))



### Split by condition


In [ ]:
DimPlot(metabo_obj, reduction = "tsne", group.by="orig.ident", split.by = "condition")
DimPlot(metabo_obj, reduction = "tsne", group.by = "orig.ident", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(metabo_obj, reduction = "tsne", group.by="orig.ident", split.by = "condition")+ NoLegend()


# Defining cell clusters  
`FindNeighbors`:    

* construction of a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity).
* *input* : the previously defined dimensionality of the dataset.

`FindClusters`:   

* default : Louvain algorithm  
* iteratively group cells together, with the goal of optimizing the standard modularity function.  
* *Resolution* : setting this parameter between 0.4-1.2 typically returns good results for single-cell datasets of around 3K cells


In [ ]:
metabo_obj <- FindNeighbors(metabo_obj, reduction = "pca", dims = 1:pc)




## Save seurat object  


In [ ]:
saveRDS(metabo_obj, file = paste0("./results/rds/samples/", line, "_FindNeighbors_metabo.rds"))




## Optimize resolution  


In [ ]:
keep <- colnames(metabo_obj[[]])[! grepl("^integrated_snn_res",colnames(metabo_obj[[]]))]
metabo_obj@meta.data <- metabo_obj@meta.data[,keep]


In [ ]:
res.seurat <- metabo_obj

plan("multisession", workers = thread)
res.seurat <- FindClusters(res.seurat, resolution = 0.2)

plan("multisession", workers = thread)
res.seurat <- FindClusters(res.seurat, resolution = 0.3)

plan("multisession", workers = thread)
res.seurat <- FindClusters(res.seurat, resolution = 0.4)

plan("multisession", workers = thread)
res.seurat <- FindClusters(res.seurat, resolution = 0.5)

plan("multisession", workers = thread)
res.seurat <- FindClusters(res.seurat, resolution = 0.6)

plan("multisession", workers = thread)
res.seurat <- FindClusters(res.seurat, resolution = 0.7)

plan("multisession", workers = thread)
res.seurat <- FindClusters(res.seurat, resolution = 0.8)


In [ ]:
clustree(res.seurat, prefix = "integrated_snn_res.")



In [ ]:
plan("multisession", workers = thread)
metabo_obj <- FindClusters(metabo_obj, resolution = 0.8)

plan("multisession", workers = thread)
metabo_obj <- FindClusters(metabo_obj, resolution = 0.9)

plan("multisession", workers = thread)
metabo_obj <- FindClusters(metabo_obj, resolution = 1)

plan("multisession", workers = thread)
metabo_obj <- FindClusters(metabo_obj, resolution = 1.1)

plan("multisession", workers = thread)
metabo_obj <- FindClusters(metabo_obj, resolution = 1.2)

plan("multisession", workers = thread)
metabo_obj <- FindClusters(metabo_obj, resolution = 1.3)

plan("multisession", workers = thread)
metabo_obj <- FindClusters(metabo_obj, resolution = 1.4)


In [ ]:
clustree(metabo_obj, prefix = "integrated_snn_res.")

